In [30]:
## Use DASH to build user GUI
# %pip install dash

from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt

from presets import setmedium

In [2]:
press_data = pd.read_csv(r"C:\Users\geral\OneDrive\Documents\GitHub\TempCompliance4HIFU\src\sample_data.csv")  # sample_data == 121 x 501 double
mediumProp = setmedium.setMedium('Water',0)

In [3]:
print(mediumProp)


{'name': 'Water', 'speed': 1500, 'density': 1000, 'absCoeff': 0.025, 'specHeatCap': 4180, 'thermDiff': 1.4599999999999998e-07}


In [44]:
help(html.Div)

Help on class Div in module dash.html.Div:

class Div(dash.development.base_component.Component)
 |  Div(children=None, id=undefined, n_clicks=undefined, n_clicks_timestamp=undefined, disable_n_clicks=undefined, key=undefined, accessKey=undefined, className=undefined, contentEditable=undefined, dir=undefined, draggable=undefined, hidden=undefined, lang=undefined, role=undefined, spellCheck=undefined, style=undefined, tabIndex=undefined, title=undefined, loading_state=undefined, **kwargs)
 |  
 |  A Div component.
 |  Div is a wrapper for the <div> HTML5 element.
 |  For detailed attribute info see:
 |  https://developer.mozilla.org/en-US/docs/Web/HTML/Element/div
 |  
 |  Keyword arguments:
 |  
 |  - children (a list of or a singular dash component, string or number; optional):
 |      The children of this component.
 |  
 |  - id (string; optional):
 |      The ID of this component, used to identify dash components in
 |      callbacks. The ID needs to be unique across all of the com

In [83]:
app = Dash('tempCompliance4HIFU')

# ALLOWED_TYPES = ("text", "number", "password", "email", "search","tel", "url", "range", "hidden",)

app.layout = [
    # Title
    html.H1(children = 'Temperature Compliance for HIFU Transducers', style = {'textAlign':'center'}), 
    html.H2(children = 'By Gerald Lee', style = {'textAlign':'center'}),

    html.Div([

        
        html.Button('CALCULATE PRESSURE AND TEMPERATURE', id='button'),
        html.Div(id='message_Display',style={'textAlign':'center'}),
    ],style={'width': '99%', 'display': 'inline-block','padding': '10px 5px'}),
    

    # RIGHT Section - DISPLAYS
    html.Div([
        html.H2('Results', style ={'textAlign':'center'}), 
            dcc.Dropdown(['Pressure','Temperature'],'Pressure',id='DROP_field2D'),
            dcc.Graph(id='GRAPH_field2D',hoverData={'points': [{'customdata': 'Pressure'}]}),
            dcc.Graph(id='GRAPH_time1D'),
    ], style={'width': '60%', 'float': 'right', 'display': 'inline-block'}),

    # SPACE
    html.Div([
        html.H3(' ',style ={'textAlign':'center'})
    ],style={'width':'3%', 'float': 'right', 'display': 'inline-block'}),

    # LEFT Section - INPUTS
    html.Div([     
            
        # Medium Section (Top)
        html.H3('Medium', style ={'textAlign':'center'}), 
        dcc.Dropdown(['Custom','Water','Glycerol','Egg White','Castor Oil'],'Custom',id='DROP_medium'),
        
        html.P('Speed of Sound [m/s]', style ={'textAlign':'left'}), 
        dcc.Input(id="Speed", type='number', placeholder='Input Speed of Sound'),

        html.P('Density [kg/m^3]', style ={'textAlign':'left'}),
        dcc.Input(id="Density", type='number', placeholder='Input Density'),

        html.P('Absorption Coeffient [Np/(m*MHz^2)]', style ={'textAlign':'left'}),
        dcc.Input(id="AbsCoeff", type='number', placeholder='Input Absorption Coefficient'),
    
        html.P('Specific Heat Capacity [J/(kg*K)]', style ={'textAlign':'left'}),
        dcc.Input(id="SpecHeatCap", type='number', placeholder='Input Specific Heat Capacity'),
            
        html.P('Thermal Diffusivity [(m^2)/s]', style ={'textAlign':'left'}),
        dcc.Input(id="ThermDiff", type='number', placeholder='Input Thermal Diffusivity'),  
        

        # Trandsucer Section (Middle)
        html.H3('Transducer', style ={'textAlign':'center'}), 

        html.P('Frequency [MHz]', style ={'textAlign':'left'}),
        dcc.Input(id="Frequency", type='number', placeholder='Input Frequency'),

        html.P('F-Number', style ={'textAlign':'left'}),
        dcc.Input(id="FNum", type='number', placeholder='Input F Number'),

        html.P('Radius [mm]', style ={'textAlign':'left'}),
        dcc.Input(id="Radius", type='number', placeholder='Input Radius'),


        # Heating Section (Right)
        html.H3('Heating', style ={'textAlign':'center'}), 

        html.P('Time Heating [s]', style ={'textAlign':'left'}),
        dcc.Input(id="HeatTime", type='number', placeholder='Input Heat Time'),

        html.P('Time Cooling [s]', style ={'textAlign':'left'}),
        dcc.Input(id="CoolTime", type='number', placeholder='Input Cool Time'),

        html.P('Duty Cycle [Percentage]', style ={'textAlign':'left'}),
        dcc.Input(id="DutyCycle", type='number', placeholder='Input Duty Cycle'),


    ], style={'width': '15%', 'float': 'right', 'display': 'inline-block'}),





]


# @callback(
#     Output('GRAPH_field2D','figure'),
#     Input('DROP_field2D','value')
# )
# def update_figure(value, xaxis_column_name, yaxis_column_name,xaxis_type, yaxis_type,year_value):
#     dff = df[df['Year'] == year_value]

#     fig = plt.imshow(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
#             y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
#             hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
#             )

#     fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

#     fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

#     fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

#     fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

#     return fig

if __name__ == '__main__':
    app.run(debug=True)
